In [25]:
# PRETRAINED CLOSED EXPERIMENT
import numpy as np
# List of your subjects
subject_ids = ['sub-01', 'sub-02', 'sub-03','sub-04','sub-06','sub-07','sub-08','sub-09','sub-10','sub-11','sub-13','sub-14','sub-15','sub-16','sub-17']

# Paths
labels_dir_closed = "/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_closed/model/eyesclosed_labels"
predictions_closed = np.load(
    "/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pred/eyes_closed_predictions.npy",
    allow_pickle=True
).item()

In [24]:
evaluation_dict_closed = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pred/evaluation_dict_gaze_closed_vs_open_closed.npy", allow_pickle=True).item()
print(evaluation_dict_closed.keys())
print(evaluation_dict_closed['/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-07_DeepMReyeClosed_label_closed.npz'].keys())

dict_keys(['/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-07_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-08_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-02_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-05_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-06_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-03_DeepMReyeClosed_label_closed.npz', '/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_cl

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import os

def plot_eyesclosed_predictions(predictions, labels_dir, subject_ids=None, n_timepoints=313, cols=4, prediction_type="default"):
    """
    Plots real vs predicted eyes-closed labels for multiple subjects in a Plotly grid.
    Handles both default and fine-tuned prediction formats.
    """
    if subject_ids is None:
        subject_ids = [f.split('_')[0] for f in os.listdir(labels_dir) if f.endswith('.npy')]
    
    subplot_titles = []
    valid_subjects = []

    for sub_id in subject_ids:
       

        # --- Load prediction ---
        

        # --- Metrics ---
        try:
            corr, _ = pearsonr(labels, pred_y)
        except Exception as e:
            print(f"Skipping {sub_id}: correlation error -> {e}")
            continue
        r2 = r2_score(labels, pred_y)

        subplot_titles.append(f"{sub_id}<br>r={corr:.2f}, R²={r2:.2f}")
        valid_subjects.append(sub_id)
    
    # --- Grid layout ---
    rows = int(np.ceil(len(valid_subjects) / cols))
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles,
                        vertical_spacing=0.08, horizontal_spacing=0.05)

    subplot_idx = 0
    for sub_id in valid_subjects:
        row = subplot_idx // cols + 1
        col = subplot_idx % cols + 1

        label_files = [f for f in os.listdir(labels_dir) if f.startswith(sub_id)]
        labels = np.load(os.path.join(labels_dir, label_files[0]))[:n_timepoints, 0, 0].squeeze()

        pred_key = [k for k in predictions.keys() if sub_id in k]
        pred_y = predictions[pred_key[0]]['pred_y'][:n_timepoints]
        pred_y = np.atleast_2d(pred_y)
        if pred_y.shape[1] > 1:
            pred_y = pred_y.mean(axis=1)
        else:
            pred_y = pred_y.squeeze()

        # --- Plot ---
        fig.add_trace(go.Scatter(y=labels, mode='lines', line=dict(color='#0E1C36', width=2), showlegend=False),
                      row=row, col=col)
        fig.add_trace(go.Scatter(y=pred_y, mode='lines', line=dict(color='#5AC7DB', width=2),
                                 name='pred_y' if subplot_idx == 0 else None, showlegend=(subplot_idx == 0)),
                      row=row, col=col)

        subplot_idx += 1

    fig.update_layout(
        width=1800,
        height=320 * rows,
        title=f"Real vs Predicted (Eyes Closed Model) per Subject ({prediction_type})",
        template="simple_white",
        showlegend=True
    )

    fig.show()


In [40]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import os

def plot_eyesclosed_predictions(predictions, labels_dir, subject_ids=None, n_timepoints=313, cols=4, prediction_type='default'):
    """
    Plots real vs predicted eyes-closed labels for multiple subjects in a Plotly grid.
    
    Args:
        predictions (dict): Dictionary of predicted arrays, keys should include subject IDs.
        labels_dir (str): Path to folder containing labels (npy files per subject).
        subject_ids (list, optional): List of subject IDs to include. If None, will infer from label files.
        n_timepoints (int): Number of timepoints to plot (default 313).
        cols (int): Number of columns in subplot grid (default 4).
    """
    
    # Infer subjects from label files if not provided
    if subject_ids is None:
        subject_ids = [f.split('_')[0] for f in os.listdir(labels_dir) if f.endswith('.npy')]
    
    # Prepare subplot titles and valid subjects
    subplot_titles = []
    valid_subjects = []
    
    for sub_id in subject_ids:
         # --- Load label ---
        label_files = [f for f in os.listdir(labels_dir) if f.startswith(sub_id)]
        if not label_files:
            print(f"No label file found for {sub_id}")
            continue
        labels = np.load(os.path.join(labels_dir, label_files[0]))[:n_timepoints, 0, 0]
        labels = labels.squeeze()
        
        # Find prediction
        if prediction_type == "fine_tuned":
            pred_key = [k for k in predictions.keys() if sub_id in k]
            if not pred_key:
                print(f"No fine-tuned prediction found for {sub_id}")
                continue
            # take first fold
            pred_y = predictions[pred_key[0]]['pred_y'][:n_timepoints, 0, 0] 
        else: 
            pred_key = [k for k in predictions if sub_id in k]
            if not pred_key:
                print(f"No prediction found for {sub_id}")
                continue
            pred_y = predictions[pred_key[0]][:n_timepoints, 0]
        
        # Compute metrics
        corr, _ = pearsonr(labels, pred_y)
        r2 = r2_score(labels, pred_y)
        
        subplot_titles.append(f"{sub_id}<br>r={corr:.2f}, R²={r2:.2f}")
        valid_subjects.append(sub_id)
    
    # Prepare grid
    rows = int(np.ceil(len(valid_subjects)/cols))
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles,
                        vertical_spacing=0.08, horizontal_spacing=0.05)
    
    # Plotting
    subplot_idx = 0
    for sub_id in valid_subjects:
        row = subplot_idx // cols + 1
        col = subplot_idx % cols + 1

        # Load labels
        label_files = [f for f in os.listdir(labels_dir) if f.startswith(sub_id)]
        labels = np.load(os.path.join(labels_dir, label_files[0]))[:n_timepoints, 0, 0].squeeze()

        # Load prediction
        if prediction_type == "fine_tuned":
            pred_key = [k for k in predictions.keys() if sub_id in k]
            pred_y = predictions[pred_key[0]]['pred_y'][:n_timepoints, 0, 0].squeeze()
        else:
            pred_key = [k for k in predictions if sub_id in k]
            pred_y = predictions[pred_key[0]][:n_timepoints, 0].squeeze()

        # Plot labels
        fig.add_trace(go.Scatter(
            y=labels, mode='lines', line=dict(color='#0E1C36', width=2), showlegend=False),
            row=row, col=col
        )

        # Plot prediction
        fig.add_trace(go.Scatter(
            y=pred_y, mode='lines', line=dict(color='#5AC7DB', width=2),
            name='pred_y' if subplot_idx==0 else None, showlegend=(subplot_idx==0)),
            row=row, col=col
        )

        subplot_idx += 1
    
    # Layout
    fig.update_layout(
        width=1800,
        height=320*rows,
        title="Real vs Predicted (Eyes Closed Model) per Subject",
        template="simple_white",
        showlegend=True
    )
    
    fig.show()

In [35]:
key = list(evaluation_dict_closed.keys())[0]
print(key)
print({k: v.shape for k, v in evaluation_dict_closed[key].items()})


/scratch/mszinte/data/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pp_data_closed_vs_open_closed/sub-07_DeepMReyeClosed_label_closed.npz
{'real_y': (939, 10, 1), 'pred_y': (939, 10, 2), 'euc_pred': (939, 10)}


In [42]:
plot_eyesclosed_predictions(predictions_closed, labels_dir_closed,subject_ids=subject_ids, n_timepoints=313, cols=4)

In [41]:
plot_eyesclosed_predictions(evaluation_dict_closed, labels_dir_closed,subject_ids=subject_ids, n_timepoints=313, cols=4, prediction_type='fine_tuned')

In [51]:
predictions_closed_ftcalib = np.load(
    "/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_closed/pred/ftcalib_eyes_closed_predictions.npy",
    allow_pickle=True
).item()
plot_eyesclosed_predictions(predictions_closed_ftcalib, labels_dir_closed,subject_ids=subject_ids, n_timepoints=313, cols=4)

In [43]:
# PRETRAINED CALIB EXPERIMENT
import numpy as np
# List of your subjects
subject_ids = ['sub-02', 'sub-03','sub-04','sub-06','sub-07','sub-08','sub-09','sub-10','sub-11','sub-13','sub-14','sub-15','sub-16','sub-17']

# Paths
labels_dir_calib = "/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_calib/model/eyesclosed_labels"
predictions_calib = np.load(
    "/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_calib/pred/calib_eyes_closed_predictions.npy",
    allow_pickle=True
).item()

plot_eyesclosed_predictions(predictions_calib, labels_dir_calib,subject_ids=subject_ids, n_timepoints=154, cols=4)

In [45]:
evaluation_dict_calib = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/int_deepmreye/deepmreye_calib/pred/evaluation_dict_gaze_closed_vs_open_calib.npy", allow_pickle=True).item()
plot_eyesclosed_predictions(evaluation_dict_calib , labels_dir_calib,subject_ids=subject_ids, n_timepoints=154, cols=4, prediction_type='fine_tuned')

In [80]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os

task_TRs = {
    "task_1": (5, 77),
    "task_2": (82, 154),
    "task_3": (159, 231),
    "task_4": (236, 308),
}

from scipy.stats import pearsonr

def run_subjectwise_taskwise_analysis_df(evaluation_dict, labels_dir, model, thresholds=[0.3], subject_ids=None):
    all_results = []
    conf_matrices = {}
    
    if subject_ids is None:
        subject_ids = [f.split('_')[0] for f in os.listdir(labels_dir) if f.endswith('.npy')]
    
    for sub_id in subject_ids:
        label_files = [f for f in os.listdir(labels_dir) if f.startswith(sub_id) and f.endswith('.npy')]
        if not label_files:
            print(f"No label file found for {sub_id}")
            continue
        
        real_y = np.load(os.path.join(labels_dir, label_files[0]))
        pred_key = [k for k in evaluation_dict if sub_id in k]
        if not pred_key:
            print(f"No prediction found for {sub_id}")
            continue
        
        pred_y = evaluation_dict[pred_key[0]]['pred_y'] if isinstance(evaluation_dict[pred_key[0]], dict) else evaluation_dict[pred_key[0]]
        
        # Continuous time series extraction
        real_closure = real_y[:, 0, 0] if len(real_y.shape) == 3 else (real_y[:, 0] if real_y.shape[1] > 1 else real_y.flatten())
        pred_closure = np.mean(pred_y[..., 0], axis=1) if len(pred_y.shape) == 3 else (pred_y[:, 0] if len(pred_y.shape) == 2 else pred_y.flatten())
        
        min_len = min(len(real_closure), len(pred_closure))
        real_closure = real_closure[:min_len]
        pred_closure = pred_closure[:min_len]
        
        valid_idx = ~(np.isnan(real_closure) | np.isnan(pred_closure))
        real_closure = real_closure[valid_idx]
        pred_closure = pred_closure[valid_idx]
        
        if len(real_closure) == 0:
            print(f"No valid data for {sub_id}")
            continue
        
        for threshold in thresholds:
            real_binary = (real_closure >= threshold).astype(int)
            pred_binary = (pred_closure >= threshold).astype(int)
            
            for task, (start, end) in task_TRs.items():
                run_length = 313
                n_runs = len(real_binary) // run_length
                
                task_real = []
                task_pred = []
                run_corrs = []
                
                for r in range(n_runs):
                    offset = r * run_length
                    r_data = real_binary[offset + start: offset + end + 1]
                    p_data = pred_binary[offset + start: offset + end + 1]
                    task_real.append(r_data)
                    task_pred.append(p_data)
                    
                    # Continuous correlation
                    r_cont = real_closure[offset + start: offset + end + 1]
                    p_cont = pred_closure[offset + start: offset + end + 1]
                    if len(r_cont) > 1:
                        corr, _ = pearsonr(r_cont, p_cont)
                        run_corrs.append(corr)
                
                task_real = np.concatenate(task_real)
                task_pred = np.concatenate(task_pred)
                
                acc = accuracy_score(task_real, task_pred)
                precision = precision_score(task_real, task_pred, zero_division=0)
                hit_rate = recall_score(task_real, task_pred, zero_division=0)
                f1 = f1_score(task_real, task_pred, zero_division=0)
                
                cm = confusion_matrix(task_real, task_pred)
                conf_matrices[(sub_id, task, threshold)] = cm
                
                majority_class = int(np.mean(task_real >= 0.5) >= 0.5)
                baseline_acc = np.mean(task_real == majority_class)
                
                # Average correlation across runs
                mean_corr = np.mean(run_corrs) if run_corrs else np.nan
                
                all_results.append({
                    'subject': sub_id,
                    'model': model,
                    'task': task,
                    'threshold': threshold,
                    'accuracy': acc,
                    'hit_rate': hit_rate,
                    'precision': precision,
                    'f1_score': f1,
                    'baseline_accuracy': baseline_acc,
                    'corr': mean_corr  
                })
    
    results_df = pd.DataFrame(all_results)
    return results_df, conf_matrices


In [85]:
task_results_df_ptclosed, task_conf_matrices_ptclosed  = run_subjectwise_taskwise_analysis_df(predictions_closed, labels_dir_closed, model = "ptclosed", thresholds=[0.3], subject_ids=subject_ids)
task_results_df_closed, task_conf_matrices_closed  = run_subjectwise_taskwise_analysis_df(evaluation_dict_closed, labels_dir_closed, model = "closed", thresholds=[0.3], subject_ids=subject_ids)
task_results_df_closed_ftcalib, task_conf_matrices_closed_ftcalib  = run_subjectwise_taskwise_analysis_df(predictions_closed_ftcalib, labels_dir_closed, model = "closed_ftcalib", thresholds=[0.3], subject_ids=subject_ids)

/Users/sinakling/softwares/anaconda3/envs/skling/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/sinakling/softwares/anaconda3/envs/skling/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/sinakling/softwares/anaconda3/envs/skling/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/sinakling/softwares/anaconda3/envs/skling/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning:

A

In [63]:
df_accuracies = pd.DataFrame()
df_accuracies = pd.concat([task_results_df_ptclosed, task_results_df_closed,task_results_df_closed_ftcalib], ignore_index=True)
print(df_accuracies.groupby(['model', 'task'])['accuracy']
    .mean())

model           task  
closed          task_1    0.627202
                task_2    0.709393
                task_3    0.688845
                task_4    0.997065
closed_ftcalib  task_1    0.773973
                task_2    0.526419
                task_3    0.713307
                task_4    0.664384
ptclosed        task_1    0.227984
                task_2    0.805284
                task_3    0.285714
                task_4    0.998043
Name: accuracy, dtype: float64


In [87]:
df_corr = pd.DataFrame()
df_corr = pd.concat([task_results_df_ptclosed, task_results_df_closed,task_results_df_closed_ftcalib], ignore_index=True)
print(df_corr.groupby(['model', 'task'])['corr']
    .mean())

model           task  
closed          task_1    0.054992
                task_2    0.883545
                task_3    0.055961
                task_4    0.623185
closed_ftcalib  task_1    0.025819
                task_2    0.689554
                task_3   -0.016542
                task_4    0.032076
ptclosed        task_1    0.035088
                task_2    0.656562
                task_3    0.052713
                task_4    0.003909
Name: corr, dtype: float64


In [82]:
from scipy.stats import permutation_test
import pandas as pd

def paired_permutation_tests(df, model_a, model_b, value):
    """
    Run paired permutation tests between two models for each task.

    Parameters:
        df (pd.DataFrame): DataFrame with columns ['subject', 'task', 'model', 'accuracy', 'corr']
        model_a (str): Name of the first model
        model_b (str): Name of the second model

    Returns:
        pd.DataFrame: A DataFrame with task, p-value, mean difference, and n_subjects
    """
    results = []

    grouped = df.groupby(['task'])
    for task, group in grouped:
        # Average across possible duplicates (subject, model)
        group_mean = group.groupby(['subject', 'model'])[value].mean().reset_index()

        # Pivot to get one row per subject, one column per model
        pivoted = group_mean[group_mean['model'].isin([model_a, model_b])] \
            .pivot(index='subject', columns='model', values=value) \
            .dropna()

        if pivoted.shape[0] < 2:
            continue  # skip if not enough subjects

        a_values = pivoted[model_a].values
        b_values = pivoted[model_b].values

        # Paired permutation test using mean difference as statistic
        result = permutation_test(
            (a_values, b_values),
            statistic=lambda x, y: (x - y).mean(),
            permutation_type='samples',
            vectorized=False,
            alternative='less',
            n_resamples=10000
        )

        results.append({
            'task': task,
            'model_a': model_a,
            'model_b': model_b,
            'mean_diff': (a_values - b_values).mean(),
            'p_value': result.pvalue,
            'n_subjects': len(pivoted)
        })

    return pd.DataFrame(results)


In [83]:
# Run paired permutation tests 
sig_df_pt_vs_closed_acc = paired_permutation_tests(df_accuracies, model_a='ptclosed', model_b='closed', value = 'accuracy')

# Run paired permutation tests 
sig_df_calib_vs_closed_acc = paired_permutation_tests(df_accuracies, model_a='closed_ftcalib', model_b='closed', value = 'accuracy')

sig_df_pt_vs_calib_acc = paired_permutation_tests(df_accuracies, model_a='ptclosed', model_b='closed_ftcalib', value = 'accuracy')

# Append the results from both pairings
sig_df_acc = pd.concat([sig_df_pt_vs_calib_acc, sig_df_pt_vs_closed_acc,sig_df_calib_vs_closed_acc], ignore_index=True)

# Show the combined dataframe with the significance results
#print(sig_df)
print(sig_df_acc)

         task         model_a         model_b  mean_diff   p_value  n_subjects
0   (task_1,)        ptclosed  closed_ftcalib  -0.545988  0.000700          14
1   (task_2,)        ptclosed  closed_ftcalib   0.278865  1.000000          14
2   (task_3,)        ptclosed  closed_ftcalib  -0.427593  0.003000          14
3   (task_4,)        ptclosed  closed_ftcalib   0.333659  1.000000          14
4   (task_1,)        ptclosed          closed  -0.399217  0.000800          14
5   (task_2,)        ptclosed          closed   0.095890  0.973603          14
6   (task_3,)        ptclosed          closed  -0.403131  0.003400          14
7   (task_4,)        ptclosed          closed   0.000978  0.808619          14
8   (task_1,)  closed_ftcalib          closed   0.146771  0.999200          14
9   (task_2,)  closed_ftcalib          closed  -0.182975  0.000200          14
10  (task_3,)  closed_ftcalib          closed   0.024462  0.892811          14
11  (task_4,)  closed_ftcalib          closed  -0.33

In [88]:
# Run paired permutation tests 
sig_df_pt_vs_closed_corr = paired_permutation_tests(df_corr, model_a='ptclosed', model_b='closed', value = 'corr')

# Run paired permutation tests 
sig_df_calib_vs_closed_corr = paired_permutation_tests(df_corr, model_a='closed_ftcalib', model_b='closed', value = 'corr')

sig_df_pt_vs_calib_corr = paired_permutation_tests(df_corr, model_a='ptclosed', model_b='closed_ftcalib', value = 'corr')

# Append the results from both pairings
sig_df_corr = pd.concat([sig_df_pt_vs_calib_corr, sig_df_pt_vs_closed_corr,sig_df_calib_vs_closed_corr], ignore_index=True)

# Show the combined dataframe with the significance results
#print(sig_df)
print(sig_df_corr)

         task         model_a         model_b  mean_diff   p_value  n_subjects
0   (task_1,)        ptclosed  closed_ftcalib   0.009269  0.610039          14
1   (task_2,)        ptclosed  closed_ftcalib  -0.032992  0.122688          14
2   (task_3,)        ptclosed  closed_ftcalib   0.069254  0.896910          14
3   (task_4,)        ptclosed  closed_ftcalib  -0.028167  0.153809          12
4   (task_1,)        ptclosed          closed  -0.019904  0.288171          14
5   (task_2,)        ptclosed          closed  -0.226983  0.000200          14
6   (task_3,)        ptclosed          closed  -0.003248  0.449855          14
7   (task_4,)        ptclosed          closed  -0.619275  0.000732          12
8   (task_1,)  closed_ftcalib          closed  -0.029173  0.174783          14
9   (task_2,)  closed_ftcalib          closed  -0.193991  0.000100          14
10  (task_3,)  closed_ftcalib          closed  -0.072503  0.053495          14
11  (task_4,)  closed_ftcalib          closed  -0.59

In [ ]:
# ACCURACY COMPARISON PLOTS
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Colors and layout
model_colors = {
    "ptclosed": "#4281A4",   # Blue
    "closed_ftcalib": "#7DBA84",    # Emerald
    "closed": "#E07A5F"    # Rust
}

# Task titles
task_titles = {
    "task_1": "Eyes Open + Vision",
    "task_2": "Eyes Blink + Vision",
    "task_3": "Eyes Open + No Vision",
    "task_4": "Eyes Closed + No Vision"
}

model_name_map = {
    "ptclosed": "DeepMReye",
    "closed_ftcalib": "DeepMReye + Calib",
    "closed": "DeepMReye + Closed"
}

# Create subplot figure: 4 tasks × 1 column
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[title for title in task_titles.values()],
    vertical_spacing=0.15
)

task_list = list(task_titles.keys())

for row, task in enumerate(task_list, start=1):
    subset = df_accuracies[df_accuracies["task"] == task]

    means = subset.groupby("model")["accuracy"].mean() * 100
    stderrs = subset.groupby("model")["accuracy"].sem() * 100

    x_vals = [model_name_map[m] for m in ["ptclosed", "closed_ftcalib", "closed"]]
    y_vals = [means.get(m, 0) for m in ["ptclosed", "closed_ftcalib", "closed"]]
    errors = [stderrs.get(m, 0) for m in ["ptclosed", "closed_ftcalib", "closed"]]
    colors = [model_colors[m] for m in ["ptclosed", "closed_ftcalib", "closed"]]

    fig.add_trace(go.Bar(
        x=x_vals,
        y=y_vals,
        error_y=dict(type='data', array=errors),
        text=[f"{v:.1f}" for v in y_vals],
        textposition="inside",
        insidetextanchor="start",
        marker_color=colors,
        width=0.5,
        showlegend=False
    ), row=row, col=1)

    # Significance checks (pairwise)
    model_list = ["ptclosed", "closed_ftcalib", "closed"]
    pair_count = 0  # counter to stagger lines

    for i, model_a in enumerate(model_list):
        for model_b in model_list[i+1:]:
            sig_row = sig_df_acc[
                (sig_df_acc.task == task) &
                (sig_df_acc.model_a == model_a) &
                (sig_df_acc.model_b == model_b)
            ]
            if not sig_row.empty and sig_row.iloc[0].p_value < 0.05:
                max_y = max(y_vals)
                # Stagger lines by pair_count
                line_y = max_y + 5 + pair_count*5
                fig.add_shape(
                    type="line",
                    x0=model_name_map[model_a],
                    x1=model_name_map[model_b],
                    y0=line_y,
                    y1=line_y,
                    line=dict(color="black", width=2),
                    xref=f"x{row}",
                    yref=f"y{row}"
                )
                pair_count += 1  # increment for next line


# Layout
fig.update_layout(
    height=1400,
    width=1000,
    title_text="All-Model Comparison Across Tasks",
    template="simple_white",
    font=dict(family="Helvetica Neue", size=15, color="black")
)

# Standardize axes
for i in range(1, 5):
    fig.update_yaxes(
        range=[0, 109],
        title_text="Classification Accuracy (%)",
        row=i, col=1
    )

fig.show()


In [99]:
# CORRELATION COMPARISON PLOTS
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Colors and layout
model_colors = {
    "ptclosed": "#4281A4",   # Blue
    "closed_ftcalib": "#7DBA84",    # Emerald
    "closed": "#E07A5F"    # Rust
}

# Task titles
task_titles = {
    "task_1": "Eyes Open + Vision",
    "task_2": "Eyes Blink + Vision",
    "task_3": "Eyes Open + No Vision",
    "task_4": "Eyes Closed + No Vision"
}

model_name_map = {
    "ptclosed": "DeepMReye",
    "closed_ftcalib": "DeepMReye + Calib",
    "closed": "DeepMReye + Closed"
}

# Create subplot figure: 4 tasks × 1 column
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[title for title in task_titles.values()],
    vertical_spacing=0.15
)

task_list = list(task_titles.keys())

for row, task in enumerate(task_list, start=1):
    subset = df_corr[df_corr["task"] == task]

    means = subset.groupby("model")["corr"].mean() 
    stderrs = subset.groupby("model")["corr"].sem() 

    x_vals = [model_name_map[m] for m in ["ptclosed", "closed_ftcalib", "closed"]]
    y_vals = [means.get(m, 0) for m in ["ptclosed", "closed_ftcalib", "closed"]]
    errors = [stderrs.get(m, 0) for m in ["ptclosed", "closed_ftcalib", "closed"]]
    colors = [model_colors[m] for m in ["ptclosed", "closed_ftcalib", "closed"]]

    fig.add_trace(go.Bar(
        x=x_vals,
        y=y_vals,
        error_y=dict(type='data', array=errors),
        text=[f"{v:.1f}" for v in y_vals],
        textposition="inside",
        insidetextanchor="start",
        marker_color=colors,
        width=0.5,
        showlegend=False
    ), row=row, col=1)

    # Significance checks (pairwise)
    model_list = ["ptclosed", "closed_ftcalib", "closed"]
    pair_count = 0  # counter to stagger lines

    for i, model_a in enumerate(model_list):
        for model_b in model_list[i+1:]:
            sig_row = sig_df_corr[
            (sig_df_corr.task.apply(lambda x: x[0]) == task) &
            (sig_df_corr.model_a == model_a) &
            (sig_df_corr.model_b == model_b)
            ]
            if not sig_row.empty and sig_row.iloc[0].p_value < 0.05:
                max_y = max(y_vals)
                # Stagger lines by pair_count
                line_y = max_y + 0.1 + pair_count*0.1
                fig.add_shape(
                    type="line",
                    x0=model_name_map[model_a],
                    x1=model_name_map[model_b],
                    y0=line_y,
                    y1=line_y,
                    line=dict(color="black", width=2),
                    xref=f"x{row}",
                    yref=f"y{row}"
                )
                pair_count += 1  


# Layout
fig.update_layout(
    height=1400,
    width=1000,
    title_text="All-Model Comparison Across Tasks",
    template="simple_white",
    font=dict(family="Helvetica Neue", size=15, color="black")
)

# Standardize axes
for i in range(1, 5):
    fig.update_yaxes(
        range=[0, 1.1],
        title_text="Correlation (r)",
        row=i, col=1
    )

fig.show()
